In [1]:
import os
import yaml
import tifffile
import numpy as np

import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))
from src import measures
from src.separator import Separator
from src.cleaner import DoubleStepCleaner

In [2]:
fpath = "/mnt/HD-LSDF/Medaka/201912_beamtime_medaka/"
db_path = "/home/ws/er5241/Repos/measuring-repo/artifacts/debug.json"

# fls = os.listdir(fpath)
# id = np.random.choice(fls)

In [3]:
def initialize(id, main_organ):
  img_path = fpath + id + "/scaled_0.5_8bit_cropped_slices.tif"
  msk_path = fpath + id + f"/{main_organ}_scaled_0.5_8bit_cropped_slices.tif"
  img = tifffile.imread(img_path)
  msk = tifffile.imread(msk_path)

  cfg_path = f"/home/ws/er5241/Repos/measuring-repo/measurement_configs/measurement/{main_organ}.yaml"
  with open(cfg_path, "r") as stream:
      configs = yaml.safe_load(stream)

  cleaning_config = configs['cleaning']
  measuring_config = configs['measures']
  centering_config = configs.get('centering', [])

  cleaner = DoubleStepCleaner(**cleaning_config)
  msk_clean, roi = cleaner(msk)
  img_clean = img[roi]

  centers = {}
  for centering in centering_config:
      centers[centering['name']] = Separator((msk_clean == centering['label_id']), centering['function'], centering['count'])

  return img_clean, msk_clean, measuring_config, centers

In [4]:
def make_measurements():
  for lbl in measuring_config:
    lbl_mask = (msk_clean == lbl['id'])
    print(lbl['id'], lbl['name'])
    for measure in lbl['measures']:
      if 'eccentricity' not in measure['function']:  # takes too long
        mf = getattr(measures, measure['function'])
        cc = centers[lbl['center']] if ('center' in lbl.keys()) else None
        measurement = mf(lbl_mask, img_clean, cc)
        print(measure['function'],measurement)
    print()    

### brain

In [ ]:
img_clean, msk_clean, measuring_config, centers = initialize(id='Medaka_1115_20-1', main_organ='brain')

In [15]:
make_measurements()

1 optical_nerves
volume [23379]
surface_area [18558]
color_mean [121.91291329825913]
color_mean_dilated [86.51276226112522]
color_median [120.0]
color_median_dilated [86.0]
color_std [19.505886426008708]
color_std_dilated [31.860549702490104]
color_perc_1 [83.78]
color_perc_1_dilated [33.0]
color_perc_99 [181.0]
color_perc_99_dilated [174.0]
convex_volume [137372.5]
radius_minimal_sphere [62.6827896126008]
2 optical_tectum
volume [183727, 181254]
surface_area [52947, 50184]
color_mean [125.06708322674403, 123.64874154501418]
color_mean_dilated [91.12837469230257, 91.71755601127613]
color_mean_eroded [123.30481980026053, 123.11895261845386]
color_median [125.0, 125.0]
color_median_dilated [83.0, 83.0]
color_median_eroded [123.0, 123.0]
color_std [14.007746193102877, 15.687701312821826]
color_std_dilated [34.267375858774756, 33.300890633424196]
color_std_eroded [7.857447258547131, 7.651744371086129]
color_perc_1 [93.0, 83.0]
color_perc_1_dilated [39.0, 42.0]
color_perc_1_eroded [106.0, 1

### heartkidney

In [ ]:
img_clean, msk_clean, measuring_config, centers = initialize(id='Medaka_1129_25-1', main_organ='heartkidney')

In [12]:
make_measurements()

2 atrium
volume [312297]
surface_area [113895]
color_mean [117.87404938247886]
color_mean_dilated [86.76456396245538]
color_mean_eroded [130.27783755583815]
color_median [120.0]
color_median_dilated [79.0]
color_median_eroded [133.0]
color_std [24.1461341750545]
color_std_dilated [32.53474937276524]
color_std_eroded [20.863448809822167]
color_perc_1 [50.0]
color_perc_1_dilated [37.0]
color_perc_1_eroded [55.0]
color_perc_99 [162.0]
color_perc_99_dilated [160.0]
color_perc_99_eroded [167.0]
convex_volume [593790.3333333333]
radius_minimal_sphere [81.1001415284453]
4 left_kidney
volume [724539]
surface_area [115294]
color_mean [119.95255879945731]
color_mean_dilated [75.15645320749333]
color_mean_eroded [118.79294249168808]
color_median [120.0]
color_median_dilated [67.0]
color_median_eroded [118.0]
color_std [21.833129345456133]
color_std_dilated [32.51693567817804]
color_std_eroded [19.50704670465549]
color_perc_1 [65.0]
color_perc_1_dilated [24.0]
color_perc_1_eroded [72.0]
color_perc

### liver

In [ ]:
img_clean, msk_clean, measuring_config, centers = initialize(id='Medaka_1115_20-1', main_organ='liver')

In [6]:
make_measurements()

1 liver
volume [957354]
surface_area [255486]
color_mean [134.92143658458627]
color_mean_dilated [106.40123660512828]
color_mean_eroded [131.5065935521058]
color_median [134.0]
color_median_dilated [110.0]
color_median_eroded [131.0]
color_std [18.96998865566971]
color_std_dilated [36.39823159943829]
color_std_eroded [15.039514243706465]
color_perc_1 [93.0]
color_perc_1_dilated [46.0]
color_perc_1_eroded [97.0]
color_perc_99 [190.0]
color_perc_99_dilated [181.0]
color_perc_99_eroded [177.0]
convex_volume [1533284.5]
radius_minimal_sphere [100.39669406502752]
eccentricity_meridional [0.2430539537752909]
eccentricity_equatorial [0.08963000738917838]



### eyes

In [5]:
img_clean, msk_clean, measuring_config, centers = initialize(id='Medaka_1115_20-1', main_organ='eyes')

In [12]:
make_measurements()

1 iris
volume [37928, 25020]
surface_area [33175, 20185]
color_mean [144.9680710820502, 151.87114308553157]
color_mean_dilated [82.94636708387488, 87.04583687787347]
color_mean_eroded [0, 0]
color_median [149.0, 154.0]
color_median_dilated [63.0, 65.0]
color_median_eroded [0, 0]
color_std [28.690803594102277, 25.435887150406366]
color_std_dilated [43.25458058451437, 45.55686397988211]
color_std_eroded [0, 0]
color_perc_1 [65.0, 75.0]
color_perc_1_dilated [42.0, 44.0]
color_perc_1_eroded [0, 0]
color_perc_99 [197.0, 202.0]
color_perc_99_dilated [199.0, 205.0]
color_perc_99_eroded [0, 0]
convex_volume [251651.99999999997, 165304.16666666666]
radius_minimal_sphere [68.06468522302916, 67.78993172389863]
distance_between_centers [285.39304728568885]

3 lens
volume [46976, 47655]
surface_area [31296, 30806]
color_mean [213.41425408719346, 221.69505823103557]
color_mean_dilated [67.0582528349433, 69.68755215435654]
color_mean_eroded [0, 0]
color_median [254.0, 255.0]
color_median_dilated [65.